In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
#df= pd.read_csv('../../data/more_features/with_correlated_feature_0.1.csv')
df =pd.read_csv('../../data/more_features/more_feats_correlated_noisy_6.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,12.322384,163.121429,4.121959,-1.00000,433.145097,100.147359,51.037057,3.691276,1,0.819482,147.693827,59.199141,41.958432,20.792161,101.383932,36.967153,11.782901,Unspecified anemia
1,8.298889,-1.000000,2.076950,-1.00000,483.617753,98.431076,-1.000000,2.529350,0,-1.000000,-1.000000,-1.000000,36.118322,-1.000000,-1.000000,24.896668,-1.000000,Hemolytic anemia
2,12.696391,3.393723,-1.000000,-1.00000,451.933132,79.486542,85.001345,4.791902,1,-1.000000,4.852168,89.831485,44.946238,0.965963,-1.000000,38.089174,18.808390,Iron deficiency anemia
3,12.705102,-1.000000,2.305379,-1.00000,-1.000000,81.057541,135.371313,4.702253,1,1.324140,32.717943,76.524319,-1.000000,27.439316,-1.000000,38.115305,-1.000000,Aplastic anemia
4,8.211543,29.622561,-1.000000,0.93619,479.914773,78.388480,-1.000000,3.142634,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,24.634629,-1.000000,Iron deficiency anemia


In [4]:
utils.get_dt_performance(df)

(0.6596428571428572,
 0.6536656960257498,
 0.8005238687826254,
 datetime.timedelta(microseconds=2998))

In [5]:
df.label.value_counts()

No anemia                               16000
Anemia of chronic disease                8828
Iron deficiency anemia                   8331
Unspecified anemia                       8104
Aplastic anemia                          8093
Hemolytic anemia                         8089
Vitamin B12/Folate deficiency anemia     8082
Inconclusive diagnosis                   4473
Name: label, dtype: int64

In [6]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [7]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [ ]:
for steps in [int(23e6), int(24e6)]:
#for steps in [int(2e3)]:
    #start_time = datetime.now()
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_6_{steps}')
    #end_time = datetime.now()
    #print(f'The duration for {steps} steps is {end_time-start_time}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.92     |
|    ep_rew_mean      | -0.9     |
|    exploration_rate | 0.885    |
|    success_rate     | 0.07     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 739      |
|    time_elapsed     | 376      |
|    total_timesteps  | 278048   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 6.32     |
|    n_updates        | 57011    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.16     |
|    ep_rew_mean      | -0.86    |
|    exploration_rate | 0.761    |
|    success_rate     | 0.1      |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 672      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.8      |
|    ep_rew_mean      | -0.98    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.04     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 529      |
|    time_elapsed     | 12193    |
|    total_timesteps  | 6452074  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.113    |
|    n_updates        | 1600518  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.64     |
|    ep_rew_mean      | -0.84    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 526      |
|    time_elapsed     | 13222    |
|    total_timesteps  | 6955604  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.81     |
|    ep_rew_mean      | -0.12    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.5      |
| time/               |          |
|    episodes         | 3100000  |
|    fps              | 480      |
|    time_elapsed     | 27269    |
|    total_timesteps  | 13091483 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.123    |
|    n_updates        | 3260370  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.92     |
|    ep_rew_mean      | -0.18    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.49     |
| time/               |          |
|    episodes         | 3200000  |
|    fps              | 478      |
|    time_elapsed     | 28164    |
|    total_timesteps  | 13475650 |
| train/              |          |
|    learning_rate  

In [23]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_6_15000000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

Using stable baselines 3
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done.....


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,3.0,0.0,1.0,1.0,0.0,"[hemoglobin, mcv, Vitamin B12/Folate deficienc...",1.0,1.0
1,5.0,1.0,0.0,-1.0,0.0,"[hemoglobin, rbc, mcv, ret_count, Aplastic ane...",5.0,6.0
2,5.0,2.0,1.0,1.0,0.0,"[hemoglobin, gender, rbc, tibc, Anemia of chro...",3.0,3.0
3,4.0,3.0,0.0,-1.0,1.0,"[hemoglobin, rbc, segmented_neutrophils, Incon...",2.0,7.0
4,4.0,4.0,0.0,-1.0,1.0,"[hemoglobin, rbc, segmented_neutrophils, Incon...",0.0,7.0


In [24]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [25]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [26]:
test_df.y_pred.value_counts()

7.0    2702
1.0    2256
3.0    1951
0.0    1943
5.0    1792
4.0    1663
6.0    1623
2.0      70
Name: y_pred, dtype: int64

In [27]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

57.90714285714286

In [28]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(4.510428571428571, 0.06085714285714286)

In [29]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.5790714285714286, 0.5369599858272509, 0.7625783161150219)

In [30]:
test_df.y_pred.unique()

array([1., 6., 3., 7., 5., 0., 4., 2.])

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)